In [1]:
import pickle as pk
import tensorflow as tf
import numpy as np
#with tf.device('/device:GPU:2'):
import keras
from keras.models import Sequential
#from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose,Activation
from keras.layers.normalization import BatchNormalization
from keras import backend as K
epochs = 5
batch_size = 128
pickle_train = open("noiseBabble_20db.pickle","rb")
x_train = pk.load(pickle_train)
pickle_trainlabel = open("cleanBabble.pickle","rb")
y_train = pk.load(pickle_trainlabel)

print("Got the input data")
x_train = np.asarray(x_train[0:100000])
y_train = np.asarray(y_train[0:100000])
print(np.asarray(x_train).shape)
#     print(x_train[0:10].shape)
X_train = []
#X_train = np.asarray(X_train)
for i in range(7,len(x_train)):
    temp = x_train[i-7]
    for j in range(1,8):
        temp = np.concatenate((temp,x_train[(i-7)+j]),axis=1)
    X_train.append(temp)
    #X_train.append(x_train[i-8:i,:].reshape(129,128))

X_train = np.asarray(X_train)
print("H",np.asarray(X_train).shape)
X_train = X_train.reshape(X_train.shape[0], 129, 128, 1)
print("H2",np.asarray(X_train).shape)
y_train = y_train.reshape(y_train.shape[0], 129, 16, 1)

model = Sequential()
model.add(Conv2D(64, kernel_size=(7, 7),strides=(1, 3),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros',input_shape=(129,128,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, (5, 5),strides=(1, 3),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(256, (1, 1),strides=(1, 3),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros'))
#model.add(Conv2DTranspose(128, (3, 3), activation='relu',padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2DTranspose(128, (3, 3),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2DTranspose(64, (5, 5),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2DTranspose(1, (7, 7),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.compile(loss=keras.losses.mean_squared_error,
              optimizer=keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999),
              metrics=['accuracy'])
#model.build()

print(model.summary())
with tf.device('/device:GPU:2') as open:
    model.fit(np.asarray(X_train), np.asarray(y_train[7:]),
              batch_size=batch_size,
              epochs=epochs,
              validation_split = 0.01
             )

model.save_weights('weights_8frames_5.h5')



Using TensorFlow backend.


Got the input data
(100000, 129, 16)
H (99993, 129, 128)
H2 (99993, 129, 128, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 123, 41, 64)       3200      
_________________________________________________________________
batch_normalization_1 (Batch (None, 123, 41, 64)       256       
_________________________________________________________________
activation_1 (Activation)    (None, 123, 41, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 119, 13, 128)      204928    
_________________________________________________________________
batch_normalization_2 (Batch (None, 119, 13, 128)      512       
_________________________________________________________________
activation_2 (Activation)    (None, 119, 13, 128)      0         
_____________________________________________________________

KeyboardInterrupt: 

In [5]:
from keras.models import load_model
#model.save('model_8frames_2.h5')

In [6]:
new_model = load_model('model_8frames_2.h5')

In [21]:
from scipy import signal
from scipy.io import wavfile
import numpy as np
sample_rate, samples = wavfile.read('/media/hd2/chaitanya/Deep_Learning/project/TCDTIMIT/Noisy_TCDTIMIT/Babble/20/volunteers/01M/straightcam/sa1.wav')
print(sample_rate)
print(len(samples))
print(samples.shape)
#samples = samples.reshape(samples.shape[0])
#samples = samples[:,1]
print((samples).shape)
NoiseBabble= []
Phase = []
f, t, Zxx = signal.stft(samples, sample_rate,nperseg=256,nfft=256)
print("Z",Zxx.shape)
phase = np.angle(Zxx)
T = len(t)//16
print("T",T)
for i in range(0,(T)*16,16):
  NoiseBabble.append(np.log(np.abs(Zxx[:,i:i+16])+1e-8))
  Phase.append(phase[:,i:i+16])
print("N",np.asarray(NoiseBabble).shape)
# if len(t) % 16 !=0:
#   NoiseBabble.append(np.log(np.abs(Zxx[:,len(t)-16:len(t)])+1e-8))
#   Phase.append(phase[:,len(t)-16:len(t)])
#   T = T+1
X_NoiseBabble = []
NoiseBabble = np.asarray(NoiseBabble)
print("S",NoiseBabble.shape)
for i in range(0,8):
    temp = NoiseBabble[0]
    for j in range(1,8):
        temp = np.concatenate((temp,NoiseBabble[j]),axis=1)
        print("tempshape",temp.shape)
    X_NoiseBabble.append(temp)
    #X_NoiseBabble = np.append(X_NoiseBabble,NoiseBabble[0:8],axis=1)
for i in range(8,len(NoiseBabble)):
    temp = NoiseBabble[i]
    for j in range(1,8):
        temp = np.concatenate((temp,NoiseBabble[(i-8)+j]),axis=1)
    X_NoiseBabble.append(temp)
    #X_NoiseBabble.append(NoiseBabble[i-8:i,:,:].reshape(129,128))
print("H1",np.asarray(X_NoiseBabble).shape)
b = np.asarray(X_NoiseBabble)
X_NoiseBabble = b.reshape(b.shape[0], 129, 128, 1)

rectified = new_model.predict(X_NoiseBabble)
rectified =rectified.reshape(rectified.shape[0], 129, 16)
print("R",rectified.shape)
A = np.zeros((129,0))
for i in range(T):
  S = np.exp(rectified[i])
  A = np.append(A,S*np.cos(Phase[i]) + 1j * S*np.sin(Phase[i]),axis=1)
t,x = signal.istft(A,nperseg=256,nfft=256)
x = np.asarray(x, dtype=np.int16)
#print(len(x))
#data=np.int16(x/np.max(np.abs(x)) * 32767)
wavfile.write('/media/hd2/chaitanya/Deep_Learning/project/TCDTIMIT/code/babble/testing/frames_2.wav', 16000, x)

16000
82944
(82944,)
(82944,)
Z (129, 649)
T 40
N (40, 129, 16)
S (40, 129, 16)
tempshape (129, 32)
tempshape (129, 48)
tempshape (129, 64)
tempshape (129, 80)
tempshape (129, 96)
tempshape (129, 112)
tempshape (129, 128)
tempshape (129, 32)
tempshape (129, 48)
tempshape (129, 64)
tempshape (129, 80)
tempshape (129, 96)
tempshape (129, 112)
tempshape (129, 128)
tempshape (129, 32)
tempshape (129, 48)
tempshape (129, 64)
tempshape (129, 80)
tempshape (129, 96)
tempshape (129, 112)
tempshape (129, 128)
tempshape (129, 32)
tempshape (129, 48)
tempshape (129, 64)
tempshape (129, 80)
tempshape (129, 96)
tempshape (129, 112)
tempshape (129, 128)
tempshape (129, 32)
tempshape (129, 48)
tempshape (129, 64)
tempshape (129, 80)
tempshape (129, 96)
tempshape (129, 112)
tempshape (129, 128)
tempshape (129, 32)
tempshape (129, 48)
tempshape (129, 64)
tempshape (129, 80)
tempshape (129, 96)
tempshape (129, 112)
tempshape (129, 128)
tempshape (129, 32)
tempshape (129, 48)
tempshape (129, 64)
tempshap

In [6]:
129*128

16512